In [82]:
%matplotlib inline 
import pandas as pd
import numpy as np
import re
import json
import matplotlib.pyplot as plt
import matplotlib
plt.style.use('ggplot')

In [2]:
data = pd.read_csv('./wuzzuf-data/data_science_dataset_wuzzuf.csv')

In [3]:
# cleaning The "city_name" column 
# my target is to add each city to same name.
# print the unique city names entered.

unique_cities = np.unique(data.city_name)
print "** there are ", len(np.unique(data.city_name)), "unique city name"
with open('./munging-data/unique_cities_unexpanded','wb') as f:
    for item in np.unique(data.city_name):
        f.write(item+"\n")

value_counts = data.city_name.value_counts()
unique_above_x = value_counts[value_counts>=3].keys()

unique_extracted = np.unique([ city.lower().strip() for city in unique_above_x ]) 
unique_extracted = [ re.split('[.,/&\-]|\sand\s|\sor\s',city,re.UNICODE) for city in unique_extracted ]

unique_list = [ s.strip() for item in unique_extracted for s in item  ]
unique_list = np.unique(unique_list)

with open('./munging-data/unique_cities_expanded','wb') as f:
    for item in unique_list:
        f.write(item+'\n')

english_pattern = re.compile('^[\d\w\s]*$')
unique_english_list = []
unique_arabic_list = []
for city in unique_list:
    if english_pattern.match(city) :
        unique_english_list.append(city)
    else:
        unique_arabic_list.append(city)  

print "** ", len(unique_list), " unique city names after filteration"
print "** ", len(unique_arabic_list), " Arabic city names"
print "** ", len(unique_english_list), " English city names"        

** there are  478 unique city name
**  104  unique city names after filteration
**  6  Arabic city names
**  98  English city names


In [4]:
with open('./munging-data/cities_names_dict.json','rb') as f:
    cities_dict = json.load(f)

In [5]:
def clean_city_name(d):
    city_name = d.city_name.lower().strip()
    cities = re.split('[.,/&\-]|\sand\s|\sor\s',city_name)
    new_cities = []
    for c in cities :
        if c == "": break;
        flag = True
        for item in cities_dict :
            for ii in cities_dict[item] :
                if re.search(ii, c) :
                    new_cities.append(item);
                    flag = False;
                    break
        if flag == True :
            new_cities.append("other('"+c.lower().strip()+"')")
    
    cities_str = ""
    for c in new_cities :
        if cities_str == "" : 
            cities_str += c
        else : 
            cities_str = cities_str +", "+c
    d.city_name = cities_str
    return d
                
new_data = data.apply(clean_city_name, axis=1)
# 297 others, and 21,533 checked

In [6]:
df2 = pd.DataFrame(columns=new_data.columns)
for x in new_data.iterrows() :
    index = x[0]
    row = x[1]
    if re.search(", ", row.city_name):
        df2.loc[len(df2)] = row
        new_data.drop(index, inplace=True)


In [7]:
indexend = len(df2)
for x in df2.iterrows():
    index = x[0]
    row = x[1]
    cities = row.city_name.split(', ')
    for city in cities :
        alt = row
        alt.city_name = city
        df2.loc[indexend] = alt
        indexend += 1 
    df2.drop(index,inplace=True)
        

In [8]:
df2.reset_index(drop=True, inplace=True)
new_data.reset_index(drop=True, inplace=True)

index = len(new_data)
for x in df2.iterrows():
    row = x[1]
    new_data.loc[index] = row
    index += 1;

In [9]:
data_otherCities_filtered = new_data.copy()
for x in data_otherCities_filtered.iterrows():
    index, row = x
    if re.search('other', row.city_name):
        data_otherCities_filtered.drop(index, inplace=True)
data_otherCities_filtered.reset_index(drop=True, inplace=True)
data_otherCities_filtered.to_csv('./wuzzuf-data/wuzzuf_data_with_cityname_cleanedandfiltered.csv',index=False)

In [30]:
data_otherCities_filtered_noDescription = data_otherCities_filtered.drop(
    ["description",'job_requirements'], axis=1) 
data_otherCities_filtered_noDescription.to_csv(
    './wuzzuf-data/wuzzuf_data_with_cityname_cleanedandfiltered_noDescription.csv', index=False)

In [ ]:
plt.

In [32]:
data_otherCities_filtered_noDescription.city_name.value_counts()

cairo               15653
giza                 4117
alexandria           1162
6th of october        225
10th of ramadan       155
mansoura              140
any                   124
ismailia               80
monufia                61
Assiut                 54
tanta                  49
suez                   40
el-obour               39
el-sharqia             30
hurghada               28
port-said              24
qalyubia               20
bani suef              19
minya                  19
el-gharbia             16
damietta               16
red sea                16
aswan                  15
behira                 14
delta                  12
dakahlia               12
remote                 11
qena                    9
el-sadat                9
el-ain el-sokhna        9
zagazig                 8
sharm el-sheikh         8
kafr el-sheikh          8
sohag                   8
fayoum                  8
south sinai             4
the canal               4
marsa alam              3
Name: city_n

In [67]:
cities_value_counts = data_otherCities_filtered_noDescription.city_name.value_counts()
cities_to_keep = cities_value_counts[ cities_value_counts.keys() == 'giza' ].keys()

data_with_cities_to_keep = data_otherCities_filtered_noDescription[ 
    data_otherCities_filtered_noDescription.city_name.isin(cities_to_keep) ]

# format post_date to year-month
def format_date(r):
    r.post_date = re.findall("^\d{4}-\d{2}", r.post_date)[0]
    return r
data_with_cities_to_keep = data_with_cities_to_keep.apply(format_date,axis=1)

data_with_cities_to_keep.to_csv('./wuzzuf-data/wuzzuf_data_with_cities_to_keep.csv',index=False)

In [ ]:
cities = [u'cairo', u'giza', u'alexandria', u'6th of october', u'10th of ramadan',
       u'mansoura', u'any']

In [93]:
data_otherCities_filtered.

0                      Entry Level
1                      Entry Level
2                      Entry Level
3                      Entry Level
4        Experienced (Non-Manager)
5        Experienced (Non-Manager)
6        Experienced (Non-Manager)
7        Experienced (Non-Manager)
8                          Manager
9                      Entry Level
10       Experienced (Non-Manager)
11       Experienced (Non-Manager)
12       Experienced (Non-Manager)
13       Experienced (Non-Manager)
14       Experienced (Non-Manager)
15                     Entry Level
16                         Manager
17       Experienced (Non-Manager)
18                     Entry Level
19       Experienced (Non-Manager)
20       Experienced (Non-Manager)
21                         Manager
22       Experienced (Non-Manager)
23                         Manager
24       Experienced (Non-Manager)
25       Experienced (Non-Manager)
26                     Entry Level
27       Experienced (Non-Manager)
28       Experienced

In [83]:
views

1408     37568.0
1389     20000.0
12064    19292.0
21732    17949.0
21733    17949.0
303      17552.0
627      16734.0
18626    13806.0
20601    13459.0
3670     12662.0
17357    12179.0
4079     12040.0
5735     11743.0
14562    11702.0
8764     11322.0
20602    10935.0
3746     10645.0
1547     10611.0
20603    10593.0
10445    10444.0
13235    10377.0
4807     10266.0
1627     10195.0
3519      9779.0
2579      9454.0
21761     9306.0
21762     9306.0
6298      9133.0
2472      9089.0
19124     9058.0
          ...   
5274         0.0
14888        0.0
14887        0.0
14881        0.0
14874        0.0
14852        0.0
14850        0.0
14845        0.0
14841        0.0
5221         0.0
14829        0.0
14713        0.0
14697        0.0
14685        0.0
5280         0.0
944          0.0
20868        0.0
5323         0.0
14479        0.0
5329         0.0
20872        0.0
960          0.0
5334         0.0
5351         0.0
14438        0.0
14433        0.0
14410        0.0
14407        0